# Тестовое задание 
## Выполнил Голенских Н.А.

### Формулировка

В качестве тестового задания предлагается 
- запрограммировать модель движения баллистической ракеты 
- осуществить её пролонгацию с помощью метода Рунге-Кутты. 

### Комментарий

В приложенных файлах есть нужные для этого **формулы**.

- ПЗ-90 – это модель Земли, принятая в ГОСТах. В файле можно почитать про **системы координат**, перевод между ними и подобное. 

- В «Константах» можно найти **значения нужных для формул констант** и формулу для **плотности атмосферы**. Предлагаю использовать формулу (8) или (9). 

- В Prolongatsia – основные формулы для осуществления **пролонгации**.
***В одной из формул может оказаться ошибка в знаке !***. 

### Цели
1. Работающий код на языке Python, запустив который можно посмотреть на результат пролонгации.
2. Презентация, содержащая графики:
    - высота от времени
    - высота от дальности
    - скорость от времени
    - скорость от дальности
    - ускорение от времени
    - ускорение от дальности
    - Траектория цели в 3D

In [1]:
import numpy as np
import matplotlib.pyplot as plt

: 

Значения констант ниже взяты из
**Егоров А.Е., Болотин Р.А. "Исходные данные по фундаментальным физическим постоянным; физическим параметрам Земли, её гравитационного поля, атмосферы и распространению радиоволн в ней. Версия 4.1"**

Параметры фигуры Земли были взяты согласно значениям в ПЗ-90.11 из табл.3.2 в документе указанном выше.

In [8]:
class Earth:
    def __init__(self):
        self.mass = 5.9724e24  # Масса Земли, кг
        self.G = 6.67408e-11  # Гравитационная постоянная, м^3/(кг*с^2)
        self.GM = 3.986004418e14 # Произведение грав. постоянной Земли на её массу, м^3/с^2
        self.omega = 7.292115e-5  # Угловая скорость вращения, рад/с
        
        self.a = 6378136 # Большая полуось эллипсоида, м
        self.p = 1 / 298.2578 # Сжатие эллипсоида, ед.
        self.b = 6356751 # Малая полуось эллипсоида, м
        self.e = 0.08181911 # Эксцентриситет, ед.
        self.c0 = -2.202095e10 # Коэффициент в формуле ускорения свободного падения, м^2
    
    def g_0(self, r: np.ndarray):
        """Рассчет ускорения свободного падения в точке в ГПСК"""
        r0 = np.linalg.norm(r)
        z_vec = np.array([0, 0, r[2]])
        z = r[2]
        return - self.GM / r0**3 * (
            r - 3 * self.c0 / r0**2 * r - 6 * self.c0 / r0**2 * z_vec + 15 * self.c0 * z**2 / r0**4 * r ) 

    def g_CA(self, r: np.ndarray):
        """Рассчет центростремительного ускорения"""
        vec = np.array([r[0], r[1], 0]) # Промежуточное представление для рассчета
        return self.omega**2 * vec

    def r_gr_loc(self, x_g: np.ndarray, phi, lambda_, H):
        """Перевод координат из ГПСК в МЗСК"""
        # Матрица направляющих косинусов
        Amg = np.array([
            [-np.cos(lambda_) * np.sin(phi), -np.sin(lambda_) * np.sin(phi), np.cos(phi)],
            [np.cos(lambda_) * np.cos(phi), np.sin(lambda_) * np.cos(phi), np.sin(phi)],
            [-np.sin(lambda_), np.cos(lambda_), 0]
        ])

        # Геоцентрическая широта начала координат МЗСК
        phi0 = np.arctan((1 - 1 / self.p)**2 * np.tan(phi))

        # Расстояние от центра Земли до проекции начала коорднат МЗСК на поверхность эллипсоида
        r = self.a / np.sqrt(1 + (1 / (1 - 1 / self.p)**2 - 1) * np.sin(phi)**2)

        # Вычисление координат центра МЗСК в ГПСК 
        x0 = np.array([
            r * np.cos(phi0) * np.cos(lambda_) + H * np.cos(phi) * np.cos(lambda_),
            r * np.cos(phi0) * np.sin(lambda_) + H * np.cos(phi) * np.sin(lambda_),
            r * np.sin(phi0) + H * np.sin(phi)
        ])      

        # Перевод вектора из ГПСК в МЗСК
        x_m = np.dot(Amg, x_g - x0)

        return x_m

    def r_loc_gr(self, x_m: np.ndarray, phi, lambda_, H):
        """Перевод координат из МЗСК в ГПСК"""
        # Матрица направляющих косинусов
        Amg = np.array([
            [-np.cos(lambda_) * np.sin(phi), -np.sin(lambda_) * np.sin(phi), np.cos(phi)],
            [np.cos(lambda_) * np.cos(phi), np.sin(lambda_) * np.cos(phi), np.sin(phi)],
            [-np.sin(lambda_), np.cos(lambda_), 0]
        ])

        # Геоцентрическая широта начала координат МЗСК
        phi0 = np.arctan((1 - 1 / self.p)**2 * np.tan(phi))

        # Расстояние от центра Земли до проекции начала коорднат МЗСК на поверхность эллипсоида
        r = self.a / np.sqrt(1 + (1 / (1 - 1 / self.p)**2 - 1) * np.sin(phi)**2)

        # Вычисление координат центра МЗСК в ГПСК 
        x0 = np.array([
            r * np.cos(phi0) * np.cos(lambda_) + H * np.cos(phi) * np.cos(lambda_),
            r * np.cos(phi0) * np.sin(lambda_) + H * np.cos(phi) * np.sin(lambda_),
            r * np.sin(phi0) + H * np.sin(phi)
        ])      

        # Перевод вектора из ГПСК в МЗСК
        x_g = x0 + np.dot(Amg.T, x_m)

        return x_g

    
    def v_loc_gr(self, v_loc: np.ndarray):
        """Перевод скорости из МЗСК в ГПСК"""
        pass

    def v_gr_loc(self, v_gr: np.ndarray):
        """Перевод скорости из ГПСК в МЗСК"""
        pass


В интервале высоты от 0 до 150 км следующее уравнение (8) обеспечивает
расчёт плотности с точностью не хуже 15%:

$$\rho(h) = 10^{A(h)},$$

$$где \:\: A(h) = −1.8977 × 10^{−29}ℎ^6 + 8.7419 × 10^{−24}ℎ^5 − 1.4909 × 10^{−18}ℎ^4 +
1.1591 × 10^{−13}ℎ^3 − 4.0740 × 10^{−9} ℎ^2 − 8.0463 × 10^{−6} ℎ + 2.0667 × 10^{−2}$$

Для прикидочных расчётов приведём также простую экспоненциальную формулу (9), которая обеспечивает точность не хуже 30% в интервале высоты от 0 до 100 км:

$$\rho(h) = 1.41 exp(-\frac{h}{6910 м})$$


In [9]:
class Atmosphere:
    def __init__(self, rho_0=1.41):
        self.rho_0 = rho_0  # Плотность на уровне моря, кг/м^3
    
    def density(self, h):
        """Плотность атмосферы на высоте h"""
        return self.rho_0 * np.exp(-h / 6910)
    
    # ФОРМУЛА НИЖЕ, ВОЗМОЖНО, ПЕРЕКАЧУЕТ В ДРУГОЕ МЕСТО
    def g_aer(self, h, gamma, v: np.ndarray): 
        """
        Рассчет аэродинамического ускорения
        gamma - баллистический коэффициент цели
        """
        rho = self.density(h)
        v_norm = np.linalg.norm(v)
        return -0.5 * gamma * rho * v_norm * v

In [10]:
class BallisticMissile:
    def __init__(self, earth, atmosphere, initial_state):
        self.earth = earth
        self.atmosphere = atmosphere
        self.state = np.array(initial_state)  # [x, y, z, vx, vy, vz]
        self.trajectory = [self.state.copy()]
        self.time = [0]
    
    def g_cor(self, v: np.ndarray):
        """Расчет Кориолисова ускорения"""
        omega = self.earth.omega
        return 2 * omega * np.cross([0, 0, 1], v)
    
    
    def g(self, gamma, h, v: np.ndarray, ):
        """Рассчет полного ускорения тела в ГПСК"""
        pass
    
    def derivatives(self, t, state):
        """Правые части уравнений движения"""
        x, y, z, vx, vy, vz = state
        h = np.sqrt(x**2 + y**2 + z**2) - self.earth.radius

        # Гравитация
        g = -self.earth.gravity_acceleration(h) * np.array([x, y, z]) / np.sqrt(x**2 + y**2 + z**2)
        
        # Атмосферное сопротивление
        rho = self.atmosphere.density(h)
        v = np.array([vx, vy, vz])
        v_mag = np.linalg.norm(v)
        drag = -0.5 * rho * v_mag * v  # Сила сопротивления
        
        # Сила Кориолиса
        cor = self.coriolis_force(v)
        
        # Полное ускорение
        a = g + drag + cor
        
        return np.array([vx, vy, vz, a[0], a[1], a[2]])
    
    def integrate(self, dt, t_max):
        """Численное интегрирование методом Рунге-Кутты 4-го порядка"""
        def runge_kutta_4(derivatives, state, t, dt):
            k1 = dt * derivatives(t, state)
            k2 = dt * derivatives(t + dt / 2, state + k1 / 2)
            k3 = dt * derivatives(t + dt / 2, state + k2 / 2)
            k4 = dt * derivatives(t + dt, state + k3)
            return state + (k1 + 2 * k2 + 2 * k3 + k4) / 6
        
        t = 0
        while t < t_max:
            self.state = runge_kutta_4(self.derivatives, self.state, t, dt)
            self.trajectory.append(self.state.copy())
            t += dt
            self.time.append(t)
    
    def plot_height_time(self):
        """График зависимости высоты от времени"""
        trajectory = np.array(self.trajectory)
        height = np.sqrt(trajectory[:, 0]**2 + trajectory[:, 1]**2 + trajectory[:, 2]**2) - self.earth.radius
        plt.figure(figsize=(10, 6))
        plt.plot(self.time, height, label=r"Высота (м)")
        plt.xlabel(r"Время (с)")
        plt.ylabel(r"Высота (м)")
        plt.title(r"Высота от времени")
        plt.legend()
        plt.grid()
        plt.show()



In [21]:
earth = Earth()
atmosphere = Atmosphere()
initial_state = [0, 0, earth.radius, 0, 300, 300] # [x, y, z, vx, vy, vz]
missile = BallisticMissile(earth, atmosphere, initial_state)

missile.integrate(dt=0.005, t_max=2.2)
missile.plot_height_time()